In [1]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2020-05-19 13:25:41--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  3.50MB/s    in 1.3s    

2020-05-19 13:25:43 (3.50 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

In this analysis, we using [MovieLens](https://grouplens.org/datasets/movielens/100k/) dataset provided by GroupLens. It conmprise 100,000 ratings (1-5) from 943 users on 1682 movies.

# **Read and preprocess data**

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import math

In [3]:
item_column_name = "movieId,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western"
item = pd.read_csv("ml-100k/u.item",sep='|',names=item_column_name.split(","),encoding='latin-1')
item

,movieId,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
col_to_removed = ['movie_title', 'release_date', 'video_release_date', 'IMDb_URL']
clear_item = item.drop(col_to_removed, axis=1).set_index('movieId')
clear_item

,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1679,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1680,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0


In [35]:
# the number of the kinds of topics in the whole recommendation system.
N = len(clear_item.columns)
N

19

# **Total genres of all the movies**

Let say we want to get the details of each movie's genre, we interest to know number of genres that relate to each of the movie. Below is the very first and rough idea come to my mind for get the number. It has other ways or maybe better approach to get desired result. Here we are use `sum` function to sum up the value 1 across the genre columns.

In [5]:
# convert index movieId as list
genre_num = clear_item.sum(axis = 1)
genre_num

movieId
1       3
2       3
3       1
4       3
5       3
       ..
1678    1
1679    2
1680    2
1681    1
1682    1
Length: 1682, dtype: int64

Next, we total up the number of movie for each genre.

In [6]:
num_by_genre = clear_item.sum()
df_num_of_genre = num_by_genre.to_frame()
df_num_of_genre.reset_index(inplace=True)
df_num_of_genre.columns = ['genre', 'total_number']
df_num_of_genre

,genre,total_number
0,unknown,2
1,Action,251
2,Adventure,135
3,Animation,42
4,Children,122
5,Comedy,505
6,Crime,109
7,Documentary,50
8,Drama,725
9,Fantasy,22


In [7]:
movies = genre_num.index.to_list()
len(movies)

1682

In [8]:
sum(df_num_of_genre['total_number'])

2893

# **Simple generate random result**

As a part of my recommendation research, I have to predict and recommend the movies for each test user. Since that is another complex part which is not a focus in this noebook, therefore, we just simple generate a prediction of movie list randomly for random 10 users. We use `random` function from Python and using `sample` function to make sure no repeating items in prediction list.

In [9]:
from random import randint

# random generate unique movieId lists
random_movieId = []
random_userId = random.sample(range(1,200), 10)

for i in range(10):
  random_movieId.append(random.sample(movies, 10))

x = {'userId' : random_userId, 'movieId' : random_movieId}

random_prediction = pd.DataFrame(x)
random_prediction.set_index('userId', inplace=True)
random_prediction

,movieId
userId,
32,"[1342, 409, 1666, 1546, 610, 511, 1447, 1671, ..."
175,"[755, 1586, 1105, 1273, 1577, 222, 194, 1621, ..."
9,"[1165, 32, 584, 1536, 623, 1437, 813, 1131, 22..."
30,"[834, 521, 748, 1454, 237, 150, 204, 1500, 122..."
95,"[617, 101, 1459, 85, 1289, 1167, 848, 1257, 57..."
156,"[899, 1374, 532, 744, 1556, 1001, 589, 1423, 9..."
60,"[1521, 239, 489, 605, 1653, 287, 895, 513, 353..."
48,"[58, 1349, 1225, 215, 555, 1282, 918, 1398, 15..."
62,"[181, 121, 1500, 486, 1000, 1206, 876, 228, 14..."


# **Build function for genre calculation**

We can pre-define the function for easy to use it whenever needed.

In [0]:
# this function is to check how many genre (topics) included in the movie item
def check_genre_num(movieid):
  n_genre = genre_num.loc[movieid]
  return n_genre

# this function is to get the list of genre that included in the movieId
def check_genre_list(movieid):
  movie_genres = genre_list_by_movieid['Genres'].loc[movieid]
  return movie_genres

# **Get the genre list for each movie**

In [11]:
# group table by index and creates a dict with lists of clear_item as values
df_dict = dict(list(clear_item.groupby(clear_item.index)))

# Gather all the genres that related to all movies respectively
movieid = []
genre_list = []

for u, v in df_dict.items():
    check = v.columns[(v == 1).any()]
    if len(check) > 0:
      movieid.append(u)
      genre_list.append(check.to_list())

d = {'movieId' : movieid, 'Genres' : genre_list}

# compile in DataFrame
genre_list_by_movieid = pd.DataFrame(d)
genre_list_by_movieid.set_index('movieId', inplace=True)
genre_list_by_movieid

,Genres
movieId,
1,"[Animation, Children, Comedy]"
2,"[Action, Adventure, Thriller]"
3,[Thriller]
4,"[Action, Comedy, Drama]"
5,"[Crime, Drama, Thriller]"
...,...
1678,[Drama]
1679,"[Romance, Thriller]"
1680,"[Drama, Romance]"


In [12]:
# This cell is just for checking if the process above is correct.
# It should generate same result for every execution.

# Enter any movieId from 1 to 1682
# if input movieId 5, the output should be 3
check_genre_num(5)

3

Now we can collect all the genre for each movie in result list. Note that the list output is based on all movies in the random prediction, it is not group according to userId.

In [13]:
# Collect all genres that related to each movie
genres = []
genre_per_list = []

for user in random_prediction.index:
  movies = random_prediction['movieId'].loc[user]
  for i in movies:
    genres.append(check_genre_list(i))

genres

[['Drama'],
 ['Comedy', 'Drama'],
 ['Drama'],
 ['Drama'],
 ['Musical'],
 ['Adventure', 'War'],
 ['Drama'],
 ['Drama'],
 ['Drama'],
 ['Action', 'Drama', 'Mystery'],
 ['Action', 'Adventure', 'Children', 'Fantasy', 'Sci-Fi'],
 ['Action', 'Crime', 'Drama'],
 ['Action', 'Adventure', 'Thriller'],
 ['Drama', 'Thriller'],
 ['Drama'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Comedy', 'Crime'],
 ['Thriller'],
 ['Film-Noir', 'Mystery'],
 ['Drama'],
 ['Comedy', 'Drama'],
 ['Documentary'],
 ['Children', 'Drama'],
 ['Drama'],
 ['Children', 'Comedy'],
 ['Comedy'],
 ['Documentary'],
 ['Thriller'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Comedy', 'Romance'],
 ['Horror', 'Thriller'],
 ['Drama', 'War'],
 ['Action', 'Romance', 'Thriller'],
 ['Western'],
 ['Drama', 'Romance'],
 ['Comedy', 'Drama'],
 ['Comedy', 'Sci-Fi'],
 ['Comedy'],
 ['Action'],
 ['Horror', 'Sci-Fi'],
 ['Drama'],
 ['Action', 'Adventure', 'Animation', 'Horror', 'Sci-Fi'],
 ['Musical'],
 ['Comedy'],
 ['Romance'],
 ['Comedy'],
 ['Film-Noir', 'T

After get all the genre for each movie in random prediction, we can rearrange the list to match with every 10 movies for each userId.

In [14]:
# Arrange the list to bind with the random prediction based on userId
genre_per_list = [genres[x:x+10] for x in range(0, len(genres),10)]
genre_per_list

[[['Drama'],
  ['Comedy', 'Drama'],
  ['Drama'],
  ['Drama'],
  ['Musical'],
  ['Adventure', 'War'],
  ['Drama'],
  ['Drama'],
  ['Drama'],
  ['Action', 'Drama', 'Mystery']],
 [['Action', 'Adventure', 'Children', 'Fantasy', 'Sci-Fi'],
  ['Action', 'Crime', 'Drama'],
  ['Action', 'Adventure', 'Thriller'],
  ['Drama', 'Thriller'],
  ['Drama'],
  ['Action', 'Adventure', 'Sci-Fi'],
  ['Comedy', 'Crime'],
  ['Thriller'],
  ['Film-Noir', 'Mystery'],
  ['Drama']],
 [['Comedy', 'Drama'],
  ['Documentary'],
  ['Children', 'Drama'],
  ['Drama'],
  ['Children', 'Comedy'],
  ['Comedy'],
  ['Documentary'],
  ['Thriller'],
  ['Action', 'Adventure', 'Sci-Fi'],
  ['Comedy', 'Romance']],
 [['Horror', 'Thriller'],
  ['Drama', 'War'],
  ['Action', 'Romance', 'Thriller'],
  ['Western'],
  ['Drama', 'Romance'],
  ['Comedy', 'Drama'],
  ['Comedy', 'Sci-Fi'],
  ['Comedy'],
  ['Action'],
  ['Horror', 'Sci-Fi']],
 [['Drama'],
  ['Action', 'Adventure', 'Animation', 'Horror', 'Sci-Fi'],
  ['Musical'],
  ['Comedy

In [15]:
# Add the list into random prediction column
random_prediction['genres'] = genre_per_list
random_prediction

,movieId,genres
userId,,
32,"[1342, 409, 1666, 1546, 610, 511, 1447, 1671, ...","[[Drama], [Comedy, Drama], [Drama], [Drama], [..."
175,"[755, 1586, 1105, 1273, 1577, 222, 194, 1621, ...","[[Action, Adventure, Children, Fantasy, Sci-Fi..."
9,"[1165, 32, 584, 1536, 623, 1437, 813, 1131, 22...","[[Comedy, Drama], [Documentary], [Children, Dr..."
30,"[834, 521, 748, 1454, 237, 150, 204, 1500, 122...","[[Horror, Thriller], [Drama, War], [Action, Ro..."
95,"[617, 101, 1459, 85, 1289, 1167, 848, 1257, 57...","[[Drama], [Action, Adventure, Animation, Horro..."
156,"[899, 1374, 532, 744, 1556, 1001, 589, 1423, 9...","[[Drama], [Comedy], [Romance], [Drama, War], [..."
60,"[1521, 239, 489, 605, 1653, 287, 895, 513, 353...","[[Comedy, Romance], [Crime, Drama, Sci-Fi], [F..."
48,"[58, 1349, 1225, 215, 555, 1282, 918, 1398, 15...","[[Drama], [Comedy], [Comedy], [Drama], [Drama]..."
62,"[181, 121, 1500, 486, 1000, 1206, 876, 228, 14...","[[Action, Adventure, Romance, Sci-Fi, War], [A..."


In [16]:
# This cell is check if the matching is correct
# Enter any movieId from 1 to 1682 based on the random prediction output movie list
check_genre_list(1160)

['Drama', 'Romance']

Compute diversity

For the research, one of the paper we referred is using the following equations to compute diversity of the recommendation item list.

![alt text](https://live.staticflickr.com/65535/49911126126_6d056e799d_b.jpg)


* Lu is recommendation list for user u.

* txi is the number of topics included in item xi.

* zlu is the number of total topics in recommendation list Lu.

* Div(Lu) is elaborate as follows:

![alt text](https://live.staticflickr.com/65535/49912986207_4b64453f05_m.jpg)

* Slu is is the number of different topics in recommendation
list Lu.

* H(Lu) denotes the topic distribution of recommendation list Lu:

![alt text](https://live.staticflickr.com/65535/49913042447_d6d0a97bed_m.jpg)

* Nt is the number of the kinds of topics in the whole recommendation system
* qj is the probability of the occurrence of topic j in recommendation list Lu.

The probability of occurrence of topic j is calculated as below:

![alt text](https://live.staticflickr.com/65535/49912232138_110f1383df_m.jpg)

* sLuj is the number of topic j in the set sLu




In [0]:
# get the movie list per user and return the unique genres for that list
def get_unique_genre_list_by_user(user_movies_list):

  unique_list = []
  
  for h in user_movies_list:
    for j in h:
      unique_list.append(j)
  
  gl = list(set(unique_list))

  return gl, len(gl)

In [32]:
for i in random_prediction.index:
  print(i)

32
175
9
30
95
156
60
48
62
115


In [0]:
def Div_lu(Lu):
  

In [0]:
# get the userId list in prediction result
sum_part = 0

def diversity_per_user(lu):
  for user in lu:
    movie_list = random_prediction['movieId'].loc[user]

    # total number of topics in recommendation list per user
    topic_list , zlu = get_unique_genre_list_by_user(movie_list)

    for item in movie_list:
      txi = check_genre_num(item)
      value1 = txi/zlu
      value2 = math.log(value1)
      sum_part += value1*value2

  diversity_lu = -(sum_part) * Div_lu(lu)

  return diversity_lu

In [24]:
random_prediction['genres'].loc[32]

[['Drama'],
 ['Comedy', 'Drama'],
 ['Drama'],
 ['Drama'],
 ['Musical'],
 ['Adventure', 'War'],
 ['Drama'],
 ['Drama'],
 ['Drama'],
 ['Action', 'Drama', 'Mystery']]

In [26]:
test = random_prediction['movieId'].loc[32]
test

[1342, 409, 1666, 1546, 610, 511, 1447, 1671, 1345, 809]

In [27]:
type(test)

list

In [31]:
check_genre_num(movieid)
check_genre_list(movieid)

TypeError: ignored